#### Topic Modelling 

In [39]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import random
from pprint import pprint

import gensim
from gensim import corpora, models
from gensim.models import LdaModel, LdaMulticore
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet', quiet=True)

True

In [5]:
corpus = pd.read_csv('corpus.csv')
corpus = corpus.drop_duplicates()
corpus.shape

(4911, 2)

In [6]:
corpus.head()

,Section_ID,Section_Text
0,Cite-1,authority. this part is issued pursuant to 12 ...
1,Cite-2,purpose this part prescribes standards under w...
2,Cite-3,scope. the standards set forth in this part ap...
3,Cite-4,reservation of authority. the occ may determin...
4,Cite-43,obligation issued by an obligor not possessing...


#### cleaning and lemmatizing docs

In [8]:
stemmer = SnowballStemmer("english")

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

def clean_text(text):
    text =  re.sub(r"[^a-z0-9]", " ", text.lower())   
    return text

In [11]:
preprocessed = corpus.Section_Text.apply(clean_text).apply(preprocess)

In [12]:
preprocessed[:5]

0                    [author, issu, pursuant, seventh]
1    [purpos, prescrib, standard, nation, bank, pur...
2    [scope, standard, forth, appli, nation, bank, ...
3    [reserv, author, determin, case, case, basi, n...
4    [oblig, issu, obligor, possess, general, power...
Name: Section_Text, dtype: object

In [13]:
corpus.Section_Text[:5]

0    authority. this part is issued pursuant to 12 ...
1    purpose this part prescribes standards under w...
2    scope. the standards set forth in this part ap...
3    reservation of authority. the occ may determin...
4    obligation issued by an obligor not possessing...
Name: Section_Text, dtype: object

#### Creating bow corpus

In [17]:
dictionary = gensim.corpora.Dictionary(preprocessed)

In [18]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [19]:
# creating bag-of-word corpus
bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed]

In [36]:
# bow document example
index = 4310
ex = bow_corpus[index]
for i in range(len(ex)):
    print("Word {} (\"{}\"),  Occurances - {}.".format(ex[i][0], dictionary[ex[i][0]], ex[i][1]))

Word 1 ("issu"),  Occurances - 2.
Word 4 ("bank"),  Occurances - 7.
Word 6 ("contain"),  Occurances - 1.
Word 9 ("nation"),  Occurances - 5.
Word 22 ("appli"),  Occurances - 5.
Word 27 ("feder"),  Occurances - 5.
Word 36 ("state"),  Occurances - 3.
Word 38 ("subject"),  Occurances - 1.
Word 49 ("determin"),  Occurances - 2.
Word 56 ("paragraph"),  Occurances - 1.
Word 63 ("section"),  Occurances - 6.
Word 67 ("general"),  Occurances - 2.
Word 76 ("requir"),  Occurances - 3.
Word 85 ("applic"),  Occurances - 9.
Word 91 ("combin"),  Occurances - 1.
Word 94 ("credit"),  Occurances - 1.
Word 104 ("follow"),  Occurances - 1.
Word 121 ("provis"),  Occurances - 1.
Word 122 ("public"),  Occurances - 4.
Word 128 ("statutori"),  Occurances - 1.
Word 156 ("shall"),  Occurances - 1.
Word 193 ("institut"),  Occurances - 1.
Word 214 ("rule"),  Occurances - 1.
Word 236 ("organ"),  Occurances - 2.
Word 240 ("busi"),  Occurances - 1.
Word 255 ("relat"),  Occurances - 1.
Word 271 ("cover"),  Occurances 

#### creaitng TF-IDF corpus

In [28]:
tfidf = models.TfidfModel(bow_corpus)

In [37]:
corpus_tfidf = tfidf[bow_corpus]

In [38]:
corpus_tfidf[index]

[(1, 0.08045598595140308),
 (4, 0.09896513502771173),
 (6, 0.05877374788891292),
 (9, 0.13564750133198109),
 (22, 0.19975940808941667),
 (27, 0.10374349398314156),
 (36, 0.10351053252792156),
 (38, 0.03823172690952112),
 (49, 0.06782336468808096),
 (56, 0.0276611726337707),
 (63, 0.10869137589313682),
 (67, 0.0785855822653381),
 (76, 0.06343088439992764),
 (85, 0.2779455339514064),
 (91, 0.0779753402236377),
 (94, 0.03482970439821828),
 (104, 0.03730462288261047),
 (121, 0.04982873762762001),
 (122, 0.19851449852946293),
 (128, 0.09081652819818375),
 (156, 0.026751642802779432),
 (193, 0.03221530029032839),
 (214, 0.05065014650992854),
 (236, 0.11208602557667292),
 (240, 0.04276316505928335),
 (255, 0.04814301342534641),
 (271, 0.04786797230138649),
 (274, 0.046043189191121395),
 (291, 0.1280013799551643),
 (297, 0.06606598377077982),
 (305, 0.18083662685517451),
 (314, 0.075057071384233),
 (326, 0.1731894805314762),
 (339, 0.138323649035197),
 (344, 0.037844682986903425),
 (382, 0.054

#### Topic Modelling using BOW  

In [40]:
lda_bow = LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=16)

In [50]:
for idx, topic in lda_bow.print_topics(-1):
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.056*"bank" + 0.026*"reserv" + 0.022*"section" + 0.020*"feder" + 0.017*"associ" + 0.015*"save" + 0.015*"paragraph" + 0.014*"leas" + 0.011*"capit" + 0.011*"payment"

Topic: 1 
Words: 0.020*"agreement" + 0.019*"account" + 0.017*"credit" + 0.014*"applic" + 0.014*"section" + 0.013*"inform" + 0.013*"loan" + 0.012*"requir" + 0.012*"file" + 0.011*"institut"

Topic: 2 
Words: 0.043*"bank" + 0.026*"mean" + 0.018*"compani" + 0.016*"section" + 0.015*"asset" + 0.014*"feder" + 0.012*"loan" + 0.012*"secur" + 0.011*"exposur" + 0.011*"save"

Topic: 3 
Words: 0.029*"applic" + 0.020*"account" + 0.017*"paragraph" + 0.017*"requir" + 0.016*"shall" + 0.015*"bank" + 0.015*"section" + 0.014*"invest" + 0.012*"provid" + 0.012*"notic"

Topic: 4 
Words: 0.027*"bank" + 0.021*"associ" + 0.021*"section" + 0.019*"save" + 0.018*"institut" + 0.017*"paragraph" + 0.014*"feder" + 0.014*"nation" + 0.013*"board" + 0.012*"risk"

Topic: 5 
Words: 0.033*"bank" + 0.030*"loan" + 0.016*"feder" + 0.015*"real" + 0

#### Topic Modelling using TF-IDF 

In [51]:
lda_tfidf = LdaMulticore(corpus_tfidf, num_topics=20, id2word=dictionary, passes=2, workers=16)

In [53]:
for idx, topic in lda_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}\n'.format(idx, topic))

Topic: 0 
Word: 0.008*"convers" + 0.007*"secur" + 0.007*"bank" + 0.007*"share" + 0.006*"proxi" + 0.006*"member" + 0.006*"offer" + 0.006*"feder" + 0.006*"vote" + 0.005*"person"

Topic: 1 
Word: 0.011*"author" + 0.010*"bank" + 0.009*"save" + 0.009*"associ" + 0.008*"loan" + 0.007*"feder" + 0.007*"section" + 0.007*"nation" + 0.007*"amend" + 0.006*"compani"

Topic: 2 
Word: 0.012*"author" + 0.009*"bank" + 0.008*"apprais" + 0.008*"state" + 0.008*"capit" + 0.007*"feder" + 0.007*"nation" + 0.006*"offic" + 0.006*"develop" + 0.006*"save"

Topic: 3 
Word: 0.010*"hear" + 0.009*"parti" + 0.009*"file" + 0.009*"request" + 0.008*"judg" + 0.008*"bank" + 0.007*"motion" + 0.007*"comptrol" + 0.007*"shall" + 0.007*"order"

Topic: 4 
Word: 0.008*"applic" + 0.008*"agreement" + 0.007*"institut" + 0.007*"inform" + 0.007*"disclosur" + 0.007*"bank" + 0.006*"insur" + 0.006*"save" + 0.006*"board" + 0.006*"account"

Topic: 5 
Word: 0.009*"save" + 0.008*"associ" + 0.008*"bank" + 0.008*"file" + 0.008*"feder" + 0.006*